# Changing the target variable to a continuous value and exploring fairness within regression:

In [7]:
import pandas as pd
import numpy as np
from folktables import ACSDataSource, ACSEmployment
from folk_data_preProcessor import preProcessor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from fairlearn.metrics import MetricFrame
from fairlearn.metrics import count, \
                              false_positive_rate, \
                              selection_rate
from fairlearn.reductions import GridSearch
from fairlearn.reductions import ExponentiatedGradient
from fairlearn.reductions import BoundedGroupLoss, ZeroOneLoss

In [2]:
# import the data
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)
features, label, group = ACSEmployment.df_to_pandas(acs_data)

In [4]:
# use the pre-built preprocessor
data = preProcessor(acs_data, features, "WAGP")

X_train, X_test, y_train, y_test = data[0], data[1], data[2], data[3]

# Standard Regression

In [8]:
# train a simple liner regressor and evaluate
linear = LinearRegression()
linear.fit(X_train, y_train)

yhat = linear.predict(X_test)
linear_RMSE = mean_squared_error(y_test, yhat)
print("Linear Model MAE is:", linear_RMSE)

Linear Model MAE is: 0.8718896082664559


In [9]:
# train a Random Forest regressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

yhat = rf.predict(X_test)
rf_RMSE = mean_squared_error(y_test, yhat)
print("RF Model MAE is:", rf_RMSE)

<ipython-input-9-5da5034739fc>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RF Model MAE is: 0.8442895173793801


# fairML regression: in-processing

In [10]:
features

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,30.0,14.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,8.0
1,18.0,14.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
2,69.0,17.0,1.0,17.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,9.0
3,25.0,1.0,5.0,17.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
4,31.0,18.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378812,61.0,21.0,1.0,1.0,2.0,0.0,4.0,1.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,6.0
378813,40.0,17.0,1.0,0.0,2.0,0.0,4.0,1.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,8.0
378814,43.0,16.0,1.0,1.0,2.0,0.0,5.0,1.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,8.0
378815,23.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,8.0
